In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from efficient_apriori import apriori as apriori1
import itertools as it
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
#vocab_kos = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/vocab.kos.txt", header = None)
#vocab_nips = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/vocab.nips.txt", header =  None)

In [3]:
#docword_enron = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/docword.kos.txt.gz", header = None)
#docword_enron = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/docword.nips.txt.gz", header = None)

In [4]:
vopcab = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/vocab.kos.txt", header = None)
docword = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/docword.kos.txt.gz", header = None)
n = 0.2
k = 4

data = docword[0:3]
data_NNZ = docword[3:]
data_NNZ.columns = ["NNZ"]
#data_enron_NNZ.head()

data_NNZ = data_NNZ['NNZ'].str.split(" ")

data_NNZ = pd.DataFrame(list(data_NNZ))
data_NNZ.columns = ["Doc_id","Word_id","Count"]

In [5]:
words = {}
for i in range(0,len(list(vopcab[0]))):
    words[i+1] = vopcab[0][i]

In [6]:
dict = data_NNZ.groupby('Doc_id')['Word_id'].apply(list).to_dict()

words_in_doc = list(dict.values())

# Normal Apriori

In [7]:
start_time = time.clock()

In [8]:
#oht_ary = te.fit(words_in_doc).transform(words_in_doc, sparse=True)
te = TransactionEncoder()
te_ary = te.fit(words_in_doc).transform(words_in_doc)
sparse_words_matrix = pd.SparseDataFrame(te_ary, columns=te.columns_, default_fill_value=False)

In [9]:
frequent_itemsets = apriori(sparse_words_matrix, min_support = n, max_len=k ,use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets

In [10]:
end_time = time.clock()
time_apriori = (end_time-start_time)/60

In [11]:
time_apriori

0.09255336715000002

In [12]:
#frequent_itemsets

In [13]:
frequent_itemsets[frequent_itemsets.length == k].itemsets

Series([], Name: itemsets, dtype: float64)

# Efficient aproiri

In [14]:
start_time = time.clock()

In [15]:
itemsets, rules = apriori1(words_in_doc, min_support=n,  min_confidence=1)

ValueError: `min_support` must be a number between 0 and 1.

In [ ]:
end_time = time.clock()
time_apriori = (end_time-start_time)

In [12]:
time_apriori

In [16]:
try:
    frequent_word_id = list(itemsets[k].keys())
except KeyError:
    print("No such values")

In [86]:

frequent_word = []
for i in range(len(frequent_word_id)):
    temp_word = []
    for j in range(k):
        temp_word.append(words[int(frequent_word_id[i][j])])
    frequent_word.append(temp_word)

In [89]:
frequent_word_with_len = pd.DataFrame(frequent_word, columns = list("word "+str(i+1) for i in range(k))) 

frequent_word_with_len["Freq"] = list(itemsets[k].values())